# Explore PZ for ComCam

Melissa Graham

Wed Mar 26 2025

## Introduction

Explore the PZ that have been generated for ComCam.

### Import packages

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import qp
import h5py

import lsst.afw.display as afwDisplay
from lsst.daf.butler import Butler
import lsst.geom

## Load data

Use ECDFS for this testing.

### PZ results files

Eric Charles has run PZ estimators on ComCam data.

The PZ results are stored in hdf5 files:

```
/sdf/data/rubin/shared/pz/projects/com_cam/data/gold_<flavor>/ECDFS/output_estimate_<algo>.hdf5
```

More information about flavors, algorithms, templates, and selections can be found in:

```
/sdf/data/rubin/shared/pz/users/echarles/work/rail_project_config/com_cam/com_cam.yaml
```

and `com_cam_library.yaml`.

From `com_cam.yaml` can see

```
root: /sdf/data/rubin/shared/pz
scratch_root: "{root}"
catalogs_dir: "{root}/data"
tract = 5063
```

So the input files are in
`/sdf/data/rubin/shared/pz/data/com_cam_dp1_gold/5063/`

such as `objectTable_tract.pq` and others.


Start with `flavor` = baseline and `algo` = knn.

In [ ]:
pz_results_file = '/sdf/data/rubin/shared/pz/projects/com_cam/data/gold_baseline/ECDFS/output_estimate_knn.hdf5'

In [ ]:
ensemble = qp.read(pz_results_file)

In [ ]:
ensemble.ancil

In [ ]:
# ensemble.objdata()

In [ ]:
ensemble.ancil['zmode']

In [ ]:
print(len(ensemble.ancil['zmode']))

The photometric redshifts are in an array of single-element arrays. Convert this to a numpy array.

In [ ]:
zmode = ensemble.ancil['zmode']

In [ ]:
data = zmode.T

In [ ]:
object_pz = np.asarray(data[0], dtype='float')

In [ ]:
object_pz

In [ ]:
del ensemble, zmode, data, pz_results_file

### Object IDs for PZ results

The bit of code needed to copy the `objectId` to the `qp` files does not yet exist.

For now, retrieve the `objectId` from this file:

```
/sdf/data/rubin/shared/pz/data/test/com_cam_dp1_gold_ECDFS.hdf5
```

Learn how to read hdf5 files: https://stackoverflow.com/questions/28170623/how-to-read-hdf5-files-in-python

In [ ]:
# filename = "/sdf/data/rubin/shared/pz/data/test/com_cam_dp1_gold_ECDFS.hdf5"

# with h5py.File(filename, "r") as f:
#     # Print all root level object names (aka keys) 
#     # these can be group or dataset names 
#     print("Keys: %s" % f.keys())
#     # get first object name/key; may or may NOT be a group
#     a_group_key = list(f.keys())[0]

#     # get the object type for a_group_key: usually group or dataset
#     print(type(f[a_group_key])) 

#     # If a_group_key is a group name, 
#     # this gets the object names in the group and returns as a list
#     # data = list(f[a_group_key])

#     # If a_group_key is a dataset name, 
#     # this gets the dataset values and returns as a list
#     data = list(f[a_group_key])
#     # preferred methods to get dataset values:
#     ds_obj = f[a_group_key]      # returns as a h5py dataset object
#     ds_arr = f[a_group_key][()]  # returns as a numpy array

In [ ]:
# data

In [ ]:
# ds_arr

In [ ]:
# print(len(ds_arr))

Read a few columns into numpy arrays.

The column keys are:

```
Keys: <KeysViewHDF5 ['dec', 'g_cModelMag', 'g_cModelMagErr', 'i_cModelMag', 'i_cModelMagErr', 'objectId', 'r_cModelMag', 'r_cModelMagErr', 'ra', 'u_cModelMag', 'u_cModelMagErr', 'y_cModelMag', 'y_cModelMagErr', 'z_cModelMag', 'z_cModelMagErr']>
```

In [ ]:
filename = "/sdf/data/rubin/shared/pz/data/test/com_cam_dp1_gold_ECDFS.hdf5"

In [ ]:
c = 5
with h5py.File(filename, "r") as f:
    a_group_key = list(f.keys())[c]
    data = list(f[a_group_key])
    object_ids = f[a_group_key][()]

In [ ]:
c = 6
with h5py.File(filename, "r") as f:
    a_group_key = list(f.keys())[c]
    data = list(f[a_group_key])
    object_rmag = f[a_group_key][()]

In [ ]:
c = 8
with h5py.File(filename, "r") as f:
    a_group_key = list(f.keys())[c]
    data = list(f[a_group_key])
    object_ra = f[a_group_key][()]

In [ ]:
c = 0
with h5py.File(filename, "r") as f:
    a_group_key = list(f.keys())[c]
    data = list(f[a_group_key])
    object_dec = f[a_group_key][()]

In [ ]:
print(len(object_ra))

Quick plot of object coordinates shows that duplicates were included (`detect_isPrimary` was not constrained to True).

In [ ]:
fig = plt.figure(figsize=(6, 4))
plt.plot(object_ra, object_dec, 'o', ms=1, mew=0, alpha=0.1, color='black')
plt.show()

### Additional object properties

The input file is:

`/sdf/data/rubin/shared/pz/data/com_cam_dp1_gold/5063/objectTable_tract.pq`.

This is just the same as what was the hdf5 file above.

In [ ]:
# filename = '/sdf/data/rubin/shared/pz/data/com_cam_dp1_gold/5063/objectTable_tract.pq'
# results = pd.read_parquet(filename)
# results

Go back to the butler and pull out fluxes and shape measurements like sizes.

```
repo = /repo/main
collection = LSSTComCam/runs/DRP/DP1/w_2025_10/DM-49359
skymap='lsst_cells_v1'
input collection: LSSTComCam/DP1/defaults
```

In [ ]:
# butler = Butler('/repo/main', collections="LSSTComCam/runs/DRP/DP1/w_2025_10/DM-49359")
butler = Butler('/repo/main', collections="LSSTComCam/runs/DRP/DP1/w_2025_09/DM-49235")

Size-related columns

```
refExtendedness
footprintArea
<f>_bdReB, <f>_bdReD
<f>_ixx, <f>_iyy, <f>_ixy
```

In [ ]:
columns = ['objectId', 'detect_isPrimary', 'coord_ra', 'coord_dec',
           'tract', 'patch', 
           'u_cModelFlux', 'g_cModelFlux', 'r_cModelFlux',
           'i_cModelFlux', 'z_cModelFlux', 'y_cModelFlux',
           'refExtendedness', 'footprintArea', 'r_bdReB', 'r_bdReD',
           'r_ixx', 'r_ixy', 'r_iyy']
object_table = butler.get('objectTable_tract', skymap='lsst_cells_v1', tract=5063,
                          parameters={'columns': columns})

In [ ]:
object_table

In [ ]:
tx = np.where(object_table['detect_isPrimary'])[0]
print(len(tx))

In [ ]:
# for i in range(30):
#     tx = np.where(object_table['objectId'] == object_ids[i])[0]
#     if len(tx) > 0:
#         tmp = -2.5 * np.log10(float(object_table['r_cModelFlux'][tx[0]])) + 31.4
#         print(i, len(tx),
#               np.round(object_rmag[i], 2),
#               np.round(tmp, 2),
#               object_table['detect_isPrimary'][tx[0]])
#         del tmp
#     else:
#         print(i, len(tx))
#     del tx

### Match

So far we have:
```
object_ids
object_rmags
object_ra
object_dec
object_pz
```

Create also `object_ix` which will be the object's index in the butler `object_table`.

In [ ]:
object_ix = np.zeros(len(object_ids), dtype='int') - 1

for i in range(len(object_ids)):
    tx = np.where(object_table['objectId'] == object_ids[i])[0]
    if len(tx) > 0:
        object_ix[i] = tx[0]
    del tx

tx = np.where(object_ix >= 0)[0]
print(len(tx), ' out of ', len(object_ids), ' matched.')

## Visualize PZ on deepCoadd

### Choose a patch

Pick coordinates for some nice low-z looking galaxies. Get the corresponding patch.

In [ ]:
my_ra_deg = 53.3360614
my_dec_deg = -27.8093222
my_spherePoint = lsst.geom.SpherePoint(my_ra_deg*lsst.geom.degrees,
                                       my_dec_deg*lsst.geom.degrees)
skymap = butler.get('skyMap', skymap='lsst_cells_v1')
tract = skymap.findTract(my_spherePoint)
patch = tract.findPatch(my_spherePoint)
my_tract = tract.tract_id
my_patch = patch.getSequentialIndex()
print('my_tract: ', my_tract)
print('my_patch: ', my_patch)

Visualize whereabouts this is in the tract using object coordinates.

In [ ]:
fig = plt.figure(figsize=(6, 4))
plt.plot(object_ra, object_dec,
         'o', ms=2, mew=0, alpha=0.1, color='black')
plt.plot(my_ra_deg, my_dec_deg, '*', ms=10, color='cyan')
plt.xlabel('ra')
plt.ylabel('dec')
plt.gca().invert_xaxis()
plt.show()

### Get the deepCoadd patch

In [ ]:
my_image = butler.get('deepCoadd',
                      tract=my_tract, patch=my_patch,
                      band='r', skymap='lsst_cells_v1')

In [ ]:
my_wcs = my_image.getWcs()

### Set pz bins

In [ ]:
use_pz = [0.04, 0.1, 0.2, 0.3, 0.5, 0.7, 1.0, 1.5]
use_clr = ['cyan', 'blue', 'purple', 'green',
           'yellow', 'orange', 'red', 'magenta']

### Use Firefly

In [ ]:
# afwDisplay.setDefaultBackend('firefly')
# afw_display = afwDisplay.Display(frame=1)

In [ ]:
# afw_display.mtv(my_image)

In [ ]:
# afw_display.setMaskTransparency(100)

In [ ]:
# afw_display.erase()

In [ ]:
# low_pz = 0.0
# for p in range(8):
#     tx = np.where((object_pz > low_pz) &
#                   (object_pz <= use_pz[p]) &
#                   (object_table['patch'][object_ix] == my_patch) &
#                   (object_table['detect_isPrimary'][object_ix]))[0]
#     print(low_pz, use_pz[p], len(tx),
#           use_clr[p])
#     low_pz = use_pz[p]

#     if len(tx) > 0:
#         with afw_display.Buffering():
#             for x in tx:
#                 coord = lsst.geom.SpherePoint(object_ra[x],
#                                               object_dec[x],
#                                               lsst.geom.degrees)
#                 xy = my_wcs.skyToPixel(coord)
#                 afw_display.dot('o', xy.getX(), xy.getY(),
#                                 size=20, ctype=use_clr[p])

### Use matplotlib

In [ ]:
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
low_pz = 0
for p in range(8):
    fig, ax = plt.subplots(figsize=(6, 6))
    display = afwDisplay.Display(frame=fig)
    display.scale('asinh', 'zscale')
    display.mtv(my_image.image)
    ax.set_axis_off()
    tx = np.where((object_pz > low_pz) &
                  (object_pz <= use_pz[p]) &
                  (object_table['patch'][object_ix] == my_patch) &
                  (object_table['detect_isPrimary'][object_ix]))[0]
    if len(tx) > 0:
        with display.Buffering():
            for x in tx:
                coord = lsst.geom.SpherePoint(object_ra[x],
                                              object_dec[x],
                                              lsst.geom.degrees)
                xy = my_wcs.skyToPixel(coord)
                display.dot('o', xy.getX(), xy.getY(),
                            size=20, ctype=use_clr[p])
    title = str(low_pz) + ' - ' + str(use_pz[p]) + \
            ' (' + str(len(tx)) + ')'
    plt.title(title)
    plt.show()
    low_pz = use_pz[p]
    del tx, title

Code to grid the eight plots. Too small.

In [ ]:
# fig = plt.figure(figsize=(4, 4))
# plt.colorbar('off')
# plt.axis('off')

# for p in range(8):
#     plt.sca(ax[i, j])
#     display = afwDisplay.Display(frame=fig)
#     display.scale('linear', 'zscale')
#     display.mtv(my_image.image)
#     ax[i, j].set_axis_off()
# plt.tight_layout()
# plt.show()

Nothing above jumped out as particularly worrisome, in terms of big low-z galaxies getting high redshifts.

But try looking across the whole tract for potential photo-z outliers and assess their impact.

## Explore PZ outliers

First check how many from the PZ catalog are duplicates.

In [ ]:
tx = np.where(object_table['detect_isPrimary'][object_ix])[0]
print(len(tx), ' out of ', len(object_ix), ' are non-duplicate objects')
del tx

### Too bright

Plot r mag vs. photo-z.

Objects that are 'too faint' for their PZ aren't a worry.

But objects that are 'too bright' for their PZ, those are a concern.

In [ ]:
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

$\mu = m-M$

The brightest galaxy is about $M = -22$ mag.

Draw a line to represent $m$ vs. $z$.

This is wrong due to K corrections etc. but just to see.

In [ ]:
fig = plt.figure(figsize=(6, 4))
tx = np.where((object_table['detect_isPrimary'][object_ix]) &
              (object_table['refExtendedness'][object_ix] == 1))[0]
plt.plot(object_pz[tx], object_rmag[tx],
         'o', ms=2, mew=0, alpha=0.1, color='black')

tmpz = (np.arange(300) + 1)/100
dmod = cosmo.distmod(tmpz)
yvals = dmod.value - 22.0
plt.plot(tmpz, yvals)
del tmpz, dmod, yvals

plt.xlabel('pz')
plt.ylabel('r mag')
plt.ylim([15, 30])
plt.show()
del tx

Identify the too-bright galaxies if they're a mag brighter than the line.

In [ ]:
object_flag_bright = np.zeros(len(object_ids), dtype='int')

tmpz = (np.arange(150) + 1)/100
dmod = cosmo.distmod(tmpz)
yvals = dmod.value - 22.0
for b,zbin in enumerate(tmpz):
    if b > 0:
        tx = np.where((object_pz > tmpz[b-1]) &
                      (object_pz <= tmpz[b]) &
                      (object_rmag < yvals[b]-1.0))[0]
        object_flag_bright[tx] = 1
        del tx

tx = np.where((object_pz > tmpz[-1]) &
              (object_rmag < yvals[-1]-1.0))[0]
object_flag_bright[tx] = 1
del tx
del tmpz, dmod, yvals

tx = np.where(object_flag_bright == 1)[0]
print(len(tx), ' out of ', len(object_ids), ' flagged; fraction ',
      np.round(len(tx)/len(object_ids),3))

In [ ]:
fig = plt.figure(figsize=(6, 4))
tx = np.where((object_table['detect_isPrimary'][object_ix]) &
              (object_table['refExtendedness'][object_ix] == 1))[0]
plt.plot(object_pz[tx], object_rmag[tx],
         'o', ms=2, mew=0, alpha=0.1, color='black')

tx = np.where((object_table['detect_isPrimary'][object_ix]) &
              (object_table['refExtendedness'][object_ix] == 1) &
              (object_flag_bright == 1))[0]
plt.plot(object_pz[tx], object_rmag[tx],
         'o', ms=1, mew=0, alpha=1, color='red')

plt.xlabel('pz')
plt.ylabel('r mag')
plt.ylim([15, 30])
plt.show()
del tx

### Too big

Objects that are 'too big' for their PZ are also a worry.

#### Footprint area

Is `footprintArea` an OK way to identify 'too big' objects?

For a radius = 100 kpc galaxy (a very big galaxy).

As a funtion of redshift:
* DA = angular diameter distance (Mpc/radian)
* scale = kpc / `"`
* size = diameter in `"`
* pix = diameter in pixels
* area = approx max footprint area

In [ ]:
fig = plt.figure(figsize=(6, 4))
tx = np.where((object_table['detect_isPrimary'][object_ix]) &
              (object_table['refExtendedness'][object_ix] == 1))[0]
plt.plot(object_pz[tx], np.log10(object_table['footprintArea'][object_ix[tx]]),
         'o', ms=2, mew=0, alpha=0.1, color='black')

radius = 100 # kpc
tmpz = (np.arange(300) + 1)/100
DA = cosmo.angular_diameter_distance(tmpz)
scale = 1000.0 * DA.value / (np.rad2deg(1.0) * 3600.0)
size = radius / scale
pix = size / 0.2
area = pix**2
plt.plot(tmpz, np.log10(area))
del radius, tmpz, DA, scale, size, pix, area

plt.xlabel('pz')
plt.ylabel('log footprint area')
plt.ylim([1.8, 5.0])
plt.show()
del tx

In [ ]:
object_flag_big = np.zeros(len(object_ids), dtype='int')

radius = 100 # kpc
tmpz = (np.arange(300) + 1)/100
DA = cosmo.angular_diameter_distance(tmpz)
scale = 1000.0 * DA.value / (np.rad2deg(1.0) * 3600.0)
size = radius / scale
pix = size / 0.2
area = pix**2

for b,zbin in enumerate(tmpz):
    if b > 0:
        tx = np.where((object_pz > tmpz[b-1]) &
                      (object_pz <= tmpz[b]) &
                      (object_table['footprintArea'][object_ix]
                       > area[b]))[0]
        object_flag_big[tx] = 1
        del tx
del radius, tmpz, DA, scale, size, pix, area

tx = np.where(object_flag_big == 1)[0]
print(len(tx), ' out of ', len(object_ids), ' flagged; fraction ',
      np.round(len(tx)/len(object_ids),3))

In [ ]:
fig = plt.figure(figsize=(6, 4))

tx = np.where((object_table['detect_isPrimary'][object_ix]) &
              (object_table['refExtendedness'][object_ix] == 1))[0]
plt.plot(object_pz[tx], np.log10(object_table['footprintArea'][object_ix[tx]]),
         'o', ms=2, mew=0, alpha=0.1, color='black')

tx = np.where((object_table['detect_isPrimary'][object_ix]) &
              (object_table['refExtendedness'][object_ix] == 1) &
              (object_flag_big == 1))[0]
plt.plot(object_pz[tx], np.log10(object_table['footprintArea'][object_ix[tx]]),
         'o', ms=1, mew=0, alpha=1, color='red')

plt.xlabel('pz')
plt.ylabel('log footprint area')
plt.ylim([1.8, 5.0])
plt.show()
del tx

#### Model profile fits

`r_bdReB` / `r_bdReD` are the Half-light ellipse of the de Vaucouleurs / exponential component fits in pixels squared.

In [ ]:
tx = np.where((object_table['detect_isPrimary'][object_ix]) &
              (object_table['refExtendedness'][object_ix] == 1))[0]
fig = plt.figure(figsize=(6, 4))
plt.plot(object_pz[tx], np.log10(object_table['r_bdReB'][object_ix[tx]]),
         'o', ms=2, mew=0, alpha=0.1, color='black')
plt.axvline(0.05, alpha=0.5)
plt.xlabel('pz')
plt.ylabel('log r_bdReB')
plt.show()
fig = plt.figure(figsize=(6, 4))
plt.plot(object_pz[tx], np.log10(object_table['r_bdReD'][object_ix[tx]]),
         'o', ms=2, mew=0, alpha=0.1, color='black')
plt.axvline(0.05, alpha=0.5)
plt.xlabel('pz')
plt.ylabel('log r_bdReD')
plt.show()
del tx

### Mark too bright and too big on patches

First identify the patches that have a lot of outliers.

In [ ]:
tx = np.where((object_flag_bright == 1) & (object_flag_big == 1))[0]
print(len(tx))

values, counts = np.unique(object_table['patch'][object_ix[tx]], return_counts=True)
sx = np.flip(np.argsort(counts))
for i in range(10):
    print(values[sx[i]], counts[sx[i]])

use_patches = values[sx[range(10)]]
print(use_patches)
del tx, values, counts, sx

Display each patch and mark the too big and too bright on them.

In [ ]:
for use_patch in use_patches:
    my_image = butler.get('deepCoadd',
                      tract=my_tract, patch=use_patch,
                      band='r', skymap='lsst_cells_v1')
    my_wcs = my_image.getWcs()

    fig, ax = plt.subplots(figsize=(8, 8))
    display = afwDisplay.Display(frame=fig)
    display.scale('asinh', 'zscale')
    display.mtv(my_image.image)
    ax.set_axis_off()

    tx1 = np.where((object_flag_bright == 1) & 
                   (object_flag_big == 0) &
                   (object_table['patch'][object_ix] == use_patch) &
                   (object_table['detect_isPrimary'][object_ix]))[0]
    tx2 = np.where((object_flag_bright == 0) & 
                   (object_flag_big == 1) &
                   (object_table['patch'][object_ix] == use_patch) &
                   (object_table['detect_isPrimary'][object_ix]))[0]
    tx3 = np.where((object_flag_bright == 1) & 
                  (object_flag_big == 1) &
                  (object_table['patch'][object_ix] == use_patch) &
                  (object_table['detect_isPrimary'][object_ix]))[0]
    with display.Buffering():
        if len(tx1) > 0:
            print('cyan is too bright, but not too big')
            for x in tx1:
                coord = lsst.geom.SpherePoint(object_ra[x], object_dec[x],
                                              lsst.geom.degrees)
                xy = my_wcs.skyToPixel(coord)
                display.dot('o', xy.getX(), xy.getY(), size=40, ctype='cyan')
        if len(tx2) > 0:
            print('yellow is too big, but not too bright')
            for x in tx2:
                coord = lsst.geom.SpherePoint(object_ra[x], object_dec[x],
                                              lsst.geom.degrees)
                xy = my_wcs.skyToPixel(coord)
                display.dot('o', xy.getX(), xy.getY(), size=40, ctype='yellow')
        if len(tx3) > 0:
            print('majenta is too big and too bright')
            for x in tx3:
                coord = lsst.geom.SpherePoint(object_ra[x], object_dec[x],
                                              lsst.geom.degrees)
                xy = my_wcs.skyToPixel(coord)
                display.dot('o', xy.getX(), xy.getY(), size=40, ctype='magenta')
    del tx1, tx2, tx3
    
    plt.title('patch: ' + str(use_patch))
    plt.show()